In [3]:
import psycopg2

# AWS RDS credentials
host = 'flowise-chatbot-database-instance-1.cifjclcixjjs.us-east-2.rds.amazonaws.com'
database = 'postgres'
user = 'postgres'
password = 'ds87398HFAERbbbvyufindsdfghyui'
port = "5432"

# Initialize cursor and connection variables
cursor = None
conn = None

try:
    # Connect to PostgreSQL database
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    print("Connected to the database successfully")

    # Create a cursor object
    cursor = conn.cursor()

    # Query to select all data from the 'products' table
    cursor.execute("SELECT * FROM user_reviews")
    # Fetch all rows
    rows = cursor.fetchall()
    
    # Get the column names
    column_names = [desc[0] for desc in cursor.description]

    # Print column names
    print(f"Columns: {column_names}")
    
    # Print each row
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection if they were created
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

    print("Connection closed.")


Connected to the database successfully
Columns: ['rating', 'title', 'text', 'small_image_url', 'medium_image_url', 'large_image_url', 'asin', 'parent_asin', 'user_id', 'timestamp', 'verified_purchase', 'helpful_vote']
Connection closed.


In [1]:
import psycopg2
from psycopg2 import sql

# AWS RDS credentials
host = 'flowise-chatbot-database-instance-1.cifjclcixjjs.us-east-2.rds.amazonaws.com'
database = 'postgres'
user = 'postgres'
password = 'ds87398HFAERbbbvyufindsdfghyui'
PORT = "5432"
# Initialize cursor and connection variables
cursor = None
conn = None

# Connect to the PostgreSQL database
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    print("Connected to the database successfully")

    # Create a cursor object
    cursor = conn.cursor()

    # SQL Query to create the table
    create_table_query = """
CREATE TABLE IF NOT EXISTS products (
    main_category VARCHAR(255),
    title VARCHAR(255),
    average_rating FLOAT CHECK (average_rating BETWEEN 0 AND 5),
    rating_number INT,
    features TEXT, -- Storing as JSON string for flexibility
    description TEXT, -- Storing as JSON string for flexibility
    price FLOAT,
    images TEXT, -- Storing as JSON string for flexibility
    videos TEXT, -- Storing as JSON string for flexibility
    store VARCHAR(255),
    categories TEXT, -- Storing as JSON string for flexibility
    details JSONB, -- Using JSONB for structured product details
    parent_asin VARCHAR(20),
    bought_together TEXT, -- Storing as JSON string for flexibility
    PRIMARY KEY (parent_asin, title)
);
"""


    # Execute the query to create the table
    cursor.execute(create_table_query)
    conn.commit()

    print("Table created successfully")

except Exception as error:
    print(f"Error: {error}")

finally:
    # Close the cursor and connection if they were created
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()
    print("Connection closed")


Connected to the database successfully
Table created successfully
Connection closed


In [2]:
import psycopg2

# AWS RDS credentials
host = 'flowise-chatbot-database-instance-1.cifjclcixjjs.us-east-2.rds.amazonaws.com'
database = 'postgres'
user = 'postgres'
password = 'ds87398HFAERbbbvyufindsdfghyui'
port = "5432"

# Initialize cursor and connection variables
cursor = None
conn = None

try:
    # Connect to PostgreSQL database
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    print("Connected to the database successfully")

    # Create a cursor object
    cursor = conn.cursor()

    # Query to select all data from the 'products' table
    cursor.execute("SELECT * FROM products")
    # Fetch all rows
    rows = cursor.fetchall()
    
    # Get the column names
    column_names = [desc[0] for desc in cursor.description]

    # Print column names
    print(f"Columns: {column_names}")
    
    # Print each row
    for row in rows:
        print(row)

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection if they were created
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

    print("Connection closed.")


Connected to the database successfully
Columns: ['main_category', 'title', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images', 'videos', 'store', 'categories', 'details', 'parent_asin', 'bought_together']
Connection closed.


In [ ]:
import psycopg2
import json

# AWS RDS credentials
host = 'flowise-chatbot-database-instance-1.cifjclcixjjs.us-east-2.rds.amazonaws.com'
database = 'postgres'
user = 'postgres'
password = 'ds87398HFAERbbbvyufindsdfghyui'
port = "5432"

# JSON file containing data
json_file = 'All_Beauty.jsonl'

# Connect to the PostgreSQL database
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    print("Connected to the database successfully")

    cursor = conn.cursor()

    # Load JSON data
    with open(json_file, 'r') as f:
        data = [json.loads(line) for line in f]

    # Insert all reviews into the table
    insert_query = """
    INSERT INTO user_reviews (
        rating,
        title,
        text,
        image,
        asin,
        parent_asin,
        user_id,
        timestamp,
        verified_purchase,
        helpful_vote
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (asin, user_id, "timestamp") DO NOTHING;
    """  # Skip rows that violate the unique constraint

    for review in data:
        # Convert `images` field to a JSON-formatted string
        images_json = json.dumps(review.get('images', []))

        try:
            cursor.execute(insert_query, (
                review.get('rating', None),
                review.get('title', None),
                review.get('text', None),
                images_json,  # Insert `images` as a single column
                review.get('asin', None),
                review.get('parent_asin', None),
                review.get('user_id', None),
                review.get('timestamp', None),
                review.get('verified_purchase', None),
                review.get('helpful_vote', None)
            ))
        except Exception as e:
            print(f"Error inserting the review: {review}. Error: {e}")
            conn.rollback()  # Roll back the transaction if an error occurs
        else:
            conn.commit()  # Commit after each successful insert

    print("All rows inserted successfully!")

except Exception as e:
    print(f"Database connection failed: {e}")

finally:
    # Close the connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")


In [ ]:
import psycopg2
from psycopg2 import sql
import json

# AWS RDS credentials
host = 'flowise-chatbot-database-instance-1.cifjclcixjjs.us-east-2.rds.amazonaws.com'
database = 'postgres'
user = 'postgres'
password = 'ds87398HFAERbbbvyufindsdfghyui'
PORT = "5432"
# Initialize cursor and connection variables
json_file = 'meta_All_Beauty.jsonl'
cursor = None
conn = None

# Connect to the PostgreSQL database
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )
    print("Connected to the database successfully")

    # Create a cursor object
    cursor = conn.cursor()
     # Load JSON data
    with open(json_file, 'r') as f:
        data = [json.loads(line) for line in f]

    # SQL Query to create the table
    insert_query = """
    INSERT INTO products (
    main_category,
    title,
    average_rating,
    rating_number,
    features , 
    description , 
    price ,
    images , 
    videos , 
    store ,
    categories,
    details,
    parent_asin ,
    bought_together,
);
"""


    # Execute the query to create the table
    cursor.execute(create_table_query)
    conn.commit()

    print("Table created successfully")

except Exception as error:
    print(f"Error: {error}")

finally:
    # Close the cursor and connection if they were created
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()
    print("Connection closed")
